In [1]:
clear;
clc;
close all;

In [8]:
load 'ex4data1.mat'
load 'ex4weights.mat'
whos

Variables in the current scope:

   Attr Name        Size                     Bytes  Class
   ==== ====        ====                     =====  ===== 
        Theta1     25x401                    80200  double
        Theta2     10x26                      2080  double
        X        5000x400                 16000000  double
        y        5000x1                      40000  double

Total is 2015285 elements using 16122280 bytes



### 手动计算一遍 cost function

In [97]:
[m n] = size(X);
a1 = [ones(m, 1), X];
size(a1);
z2 = a1 * Theta1';
size(z2);
a2 = [ones(m, 1), sigmoid(z2)];
size(a2)
z3 = a2 * Theta2';
size(z3)
a3 = sigmoid(z3);
% size(a3)
% a3(1:20)
% for i = 1:10
%     yMat = [y == i, yMat];
% end
yMat = [
    y == 1, y == 2, y == 3, y == 4, y == 5, y == 6, y == 7, y == 8, y == 9, y == 10
];
% size(yMat)
% randIndices = randperm(5000);
% yMat(randIndices(1:10), :);
format long
-(1/m) * sum(sum(yMat .* log(a3) + (1 - yMat) .* log(1 - a3)))

ans =

   5000     26

ans =

   5000     10

ans =  0.287629165161319


In [88]:
v = ones(3, 1)
v2 = 3 * ones(3, 1)
[v,v2]
sum([v, v2])
sum([v, v2], 2)

v =

   1
   1
   1

v2 =

   3
   3
   3

ans =

   1   3
   1   3
   1   3

ans =

   3   9

ans =

   4
   4
   4

